топ

0.973467824363582




In [38]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from optuna.integration import LightGBMPruningCallback


In [39]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
sample_submission_df = pd.read_csv("sample_submission.csv")

target_col = "type"
id_col = "idx"
features = [c for c in train_df.columns if c not in [target_col, id_col]]

X = train_df[features].copy()
y = train_df[target_col].copy()
X_test = test_df[features].copy()

# ----------------- Категориальные -----------------
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

for col in cat_cols:
    le = LabelEncoder()
    combined = pd.concat([X[col], X_test[col]], axis=0).astype(str)
    le.fit(combined)
    X[col] = le.transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))
    X[col] = X[col].astype("category")
    X_test[col] = X_test[col].astype("category")

# ----------------- Числовые -----------------
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

imputer = SimpleImputer(strategy="median")
X[num_cols] = imputer.fit_transform(X[num_cols])
X_test[num_cols] = imputer.transform(X_test[num_cols])

scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# ----------------- Цель -----------------
le_type = LabelEncoder()
y_encoded = le_type.fit_transform(y)


In [40]:
def objective(trial, x_data, y_data, label_encoder):
    params = {
        "objective": "multiclass",
        "num_class": len(label_encoder.classes_),
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 256),
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 100),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "class_weight": "balanced",
        "random_state": 42,
        "n_jobs": -1,
        "device": "gpu",
        "verbose": -1,
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in skf.split(x_data, y_data):
        X_train, X_val = x_data.iloc[train_idx], x_data.iloc[val_idx]
        y_train, y_val = y_data[train_idx], y_data[val_idx]

        model = lgb.LGBMClassifier(**params, n_estimators=5000)
        callbacks = [lgb.early_stopping(stopping_rounds=200, verbose=False)]

        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="multi_logloss",
            callbacks=callbacks
        )

        y_val_pred = model.predict(X_val)
        score = f1_score(y_val, y_val_pred, average="macro")
        scores.append(score)

    return np.mean(scores)


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial, X, y_encoded, le_type), n_trials=100, show_progress_bar=True)

print(f"Лучший Macro F1 Score: {study.best_value:.4f}")
print("Лучшие гиперпараметры:", study.best_params)


[I 2025-09-23 22:13:49,280] A new study created in memory with name: no-name-d3599af9-f836-4940-b534-9b74af4ab1dd
Best trial: 0. Best value: 0.967353:   1%|          | 1/100 [01:55<3:10:20, 115.35s/it]

[I 2025-09-23 22:15:44,633] Trial 0 finished with value: 0.9673527412693564 and parameters: {'learning_rate': 0.021849108543391206, 'num_leaves': 137, 'max_depth': 15, 'min_child_samples': 81, 'feature_fraction': 0.9727429338636502, 'bagging_fraction': 0.8448266696695111, 'bagging_freq': 7, 'lambda_l1': 1.7114893214981772, 'lambda_l2': 4.248333439344142e-05}. Best is trial 0 with value: 0.9673527412693564.


Best trial: 0. Best value: 0.967353:   2%|▏         | 2/100 [05:41<4:54:50, 180.51s/it]

[I 2025-09-23 22:19:30,756] Trial 1 finished with value: 0.9577386304638921 and parameters: {'learning_rate': 0.026601342240003156, 'num_leaves': 217, 'max_depth': 13, 'min_child_samples': 100, 'feature_fraction': 0.6364744691834523, 'bagging_fraction': 0.8326177873142135, 'bagging_freq': 7, 'lambda_l1': 2.428758112500822e-07, 'lambda_l2': 0.0005907253806134064}. Best is trial 0 with value: 0.9673527412693564.


Best trial: 0. Best value: 0.967353:   3%|▎         | 3/100 [06:34<3:18:02, 122.50s/it]

...

[I 2025-09-23 23:53:06,074] Trial 43 finished with value: 0.9742431853805963 and parameters: {'learning_rate': 0.08058796455913272, 'num_leaves': 72, 'max_depth': 10, 'min_child_samples': 71, 'feature_fraction': 0.7662396661297425, 'bagging_fraction': 0.6009787244946042, 'bagging_freq': 1, 'lambda_l1': 0.16088824866173737, 'lambda_l2': 2.5265604031112594}. Best is trial 39 with value: 0.9752389216373695.


Best trial: 39. Best value: 0.975239:  45%|████▌     | 45/100 [1:41:06<1:48:17, 118.13s/it]

[I 2025-09-23 23:54:55,668] Trial 44 finished with value: 0.9723694531848313 and parameters: {'learning_rate': 0.07977916038259832, 'num_leaves': 59, 'max_depth': 10, 'min_child_samples': 71, 'feature_fraction': 0.7036457678136802, 'bagging_fraction': 0.6282434037804285, 'bagging_freq': 1, 'lambda_l1': 0.19305440374518434, 'lambda_l2': 3.4805540699199606}. Best is trial 39 with value: 0.9752389216373695.


Best trial: 39. Best value: 0.975239:  46%|████▌     | 46/100 [1:42:09<1:31:23, 101.54s/it]

[I 2025-09-23 23:55:58,496] Trial 45 finished with value: 0.9703518078127813 and parameters: {'learning_rate': 0.06843963393517101, 'num_leaves': 94, 'max_depth': 12, 'min_child_samples': 61, 'feature_fraction': 0.7828804156341428, 'bagging_fraction': 0.6662869089214737, 'bagging_freq': 1, 'lambda_l1': 1.0525321412105513, 'lambda_l2': 0.11758475444816754}. Best is trial 39 with value: 0.9752389216373695.


In [ ]:
best_params = study.best_params
final_model = lgb.LGBMClassifier(**best_params, n_estimators=5000)
final_model.fit(X, y_encoded)

# ----------------- Предсказание -----------------
test_preds = final_model.predict(X_test)
predicted_class_names = le_type.inverse_transform(test_preds)

submission_df = pd.DataFrame({id_col: test_df[id_col]})
submission_df[target_col] = predicted_class_names

submission_df.to_csv("result.csv", index=False)